# NATURAL LANGUAGE PROCESSING
## Mini Project 1 – Text Generation

### Loading Data

#### Load khmer_food.txt

#### Display First 200 sentences

#### Split the corpus into training, validation, and test sets

### Tokenize the Text

### Exploratory Data Analysis (EDA)

#### Basic Statistics of tokens

#### Frequency Distribution

#### Visualization - Word Cloud

### Pre-Processing the Data

Complete function below which
+ Convert lower case
+ Remove emoji
+ Split each sentence into tokens
+ Remove punctuation
+ Then, generate unigrams, bigrams, trigrams, 4-grams
Hint: For unigrams, you can directly use the tokenized list.
For bigrams, use nltk.bigrams
+ This function will return two lists containing unigrams, bigrams

In [ ]:
# Write your code here


+ apply the function to the data
+ count frequency of 4-grams
+ plot 100 most common 4-grams

In [ ]:
# Write your code here

### Build 4-gram language model using backoff method

### Build 4-gram language model using interpolation method with add-k smoothing

### Evaluate perplexity of a language model on a test set

In [ ]:
# Write your code here

### Generate text using the language model

### Generate the next word given the previous tokens

## Runing Code ......

In [ ]:
# wrtie your code here 